In [58]:
import json
import read_data
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from optparse import OptionParser
import torch.autograd as autograd
from copy import deepcopy
import os
import sys
import pprint
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
torch.manual_seed(1)

from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [2]:
from datetime import datetime
start_time = datetime.now()
today = start_time.strftime('%Y-%m-%d')

In [3]:
# load data
data = read_data.load_trn_data()
trn_conll = read_data.load_trn_nlp()

In [4]:
# input data
# [
#     ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'], 
#     ['인사동에', '들어서면', '다종다양의', '창호지,', '도자기', '등', '고미술품들이', '진열장에', '즐비하게', '널려져', '있는', '것을', '볼', '수', '있다.'], 
#     ['ARGM-LOC', '-', '-', '-', '-', '-', 'ARG1', 'ARG1', '-', '-', '-', 'ARG1', '-', '-', '-']
# ]

def get_input_data(data):
    result = []
    for sent in data:
        sent_list = []
        
        tok_idx = []
        tok_str = []
        tok_arg = []
        for token in sent:
            tok_idx.append(token[0])
            tok_str.append(token[1])
            tok_arg.append(token[2])
            
        sent_list.append(tok_idx)
        sent_list.append(tok_str)
        sent_list.append(tok_arg)
        result.append(sent_list)
    return result
        
input_data = get_input_data(data)

In [68]:
div = len(input_data) - dev_sent
dev = input_data[div:]
trn = input_data[:div]
gold_file = './dev.data'

NameError: name 'dev_sent' is not defined

In [6]:
def prepare_idx():
    dp_to_ix, arg_to_ix = {},{}
    dp_to_ix['null'] = 0
    for sent in trn_conll:
        for token in sent:
            dp = token[11]
            if dp not in dp_to_ix:
                dp_to_ix[dp] = len(dp_to_ix)
    
    args = ['ARG0', 'ARG1', 'ARG2', 'ARG3', 'ARGM-CAU', 'ARGM-CND', 'ARGM-DIR', 'ARM-DIS', 'ARGM-INS', 'ARGM-LOC', 'ARCM-MNR', 'ARCM-NEG', 'ARCM-PRD', 'ARCM-PRP', 'ARCM-TMP', 'ARCM-ADV', 'ARCM-EXT', '-']
    for i in args:
        if i not in arg_to_ix:
            arg_to_ix[i] = len(arg_to_ix)
    return dp_to_ix, arg_to_ix
dp_to_ix, arg_to_ix = prepare_idx()
DP_VOCAB_SIZE = len(dp_to_ix)
ARG_VOCAB_SIZE = len(arg_to_ix)
print('DP_VOCAB_SIZE:',DP_VOCAB_SIZE)
print('ARG_VOCAB_SIZE:',ARG_VOCAB_SIZE)

DP_VOCAB_SIZE: 32
ARG_VOCAB_SIZE: 18


In [19]:
configuration = {'token_dim': 60,
                 'hidden_dim': 64,
                 'feat_dim': 2,
                 'dp_dim': 4,
                 'arg_dim': 4,
                 'lu_pos_dim': 5,
                 'dp_label_dim': 10,
                 'lstm_input_dim': 768,
                 'lstm_dim': 64,
                 'lstm_depth': 2,
                 'hidden_dim': 64,
                 'position_feature_dim': 5,
                 'num_epochs': 50,
                 'learning_rate': 0.001,
                 'dropout_rate': 0.01,
                 'pretrained_embedding_dim': 300,
                 'model_path': './result/baseline'
                 }
print('\n### CONFIGURATION ###\n')
pprint.pprint(configuration)
print('')

DPDIM = configuration['dp_dim']
ARGDIM = configuration['arg_dim']
LSTMINPDIM = configuration['lstm_input_dim']
FEATDIM = configuration['feat_dim']
HIDDENDIM = configuration['hidden_dim']
LSTMDEPTH = configuration['lstm_depth']
DROPOUT_RATE = configuration['dropout_rate']
learning_rate = configuration['learning_rate']
NUM_EPOCHS = configuration['num_epochs']
model_path = configuration['model_path']

print('\n### YOUR MODEL WILL BE SAVED TO', model_path, '###\n')


### CONFIGURATION ###

{'arg_dim': 4,
 'dp_dim': 4,
 'dp_label_dim': 10,
 'dropout_rate': 0.01,
 'feat_dim': 2,
 'hidden_dim': 64,
 'learning_rate': 0.001,
 'lstm_depth': 2,
 'lstm_dim': 64,
 'lstm_input_dim': 768,
 'lu_pos_dim': 5,
 'model_path': './result/baseline',
 'num_epochs': 50,
 'position_feature_dim': 5,
 'pretrained_embedding_dim': 300,
 'token_dim': 60}


### YOUR MODEL WILL BE SAVED TO ./result/baseline ###



In [7]:
# load BERT model

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual')
bert_model = BertModel.from_pretrained('bert-base-multilingual')

12/06/2018 15:13:22 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-vocab.txt from cache at /home/hahmyg/.pytorch_pretrained_bert/3f396e8b6d1942457b908bd7f351fa991ead4c4adef76c76189a9ace12841860.535306b226c42cebebbc0dabc83b92ab11260e9919e21e2ab0beb301f267b4c7
12/06/2018 15:13:23 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual.tar.gz from cache at /home/hahmyg/.pytorch_pretrained_bert/e359baa6e6b29d9971ac7525c83e8cad6f15dce9d8ec9bfdeafa149a7a2191c9.5e2593d7d76d4df2b618714d71af902c02a5f51c1b2d050399e1cb36b7bb2eeb
12/06/2018 15:13:23 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /home/hahmyg/.pytorch_pretrained_bert/e359baa6e6b29d9971ac7525c83e8cad6f15dce9d8ec9bfdeafa149a7a2191c9.5e2593d7d76d4df2b618714d71af902c02a5f51c1b2d050399e1cb36b7bb2eeb to temp dir /tmp/tmp09esgg9i
1

In [8]:
def tokenization(tokens):
    bert_tokens = []
    orig_to_token_map = []
    bert_tokens.append("[CLS]")
    for i in range(len(tokens)):
        origin_token = tokens[i]
        orig_to_token_map.append(len(bert_tokens))
        bert_tokens.extend(tokenizer.tokenize(origin_token))
    
    return bert_tokens, orig_to_token_map

In [9]:
def get_pred_idxs(conll):
    result = []
    preds = [0 for i in range(len(conll))]
    for i in range(len(conll)):
        tok = conll[i]
        if tok[10].startswith('V'):
            preds = [0 for item in range(len(conll))]
            preds[i] = 1
            result.append(preds)
    return result

In [10]:
def get_arg_idxs(pred_idx, conll):
    arg_idxs = [0 for i in range(len(conll))]
    for i in range(len(conll)):
        tok = conll[i]
        if int(tok[8]) == pred_idx:
            arg_pos = tok[-1]
            if arg_pos[:2] == 'NP':
                arg_idxs[i] = 1
    return arg_idxs

In [11]:
def get_feature(pred_idxs, conll):
    result = []
    for i in pred_idxs:
#         print(i)
        features = []
        for j in range(len(i)):
            pred_idx = i[j]
            if pred_idx == 1:
                arg_idxs = get_arg_idxs(j, conll)
#                 print(arg_idxs)
        for j in range(len(i)):
            feature = []                
            feature.append(i[j])
            feature.append(arg_idxs[j])
            features.append(feature)
                
        result.append(features)
    return result

In [12]:
def prepare_sequence(seq, to_ix):
    vocab = list(to_ix.keys())
    idxs = []
    for w in seq:
        if w in vocab:
            idxs.append(to_ix[w])
        else:
            idxs.append(0)  
#     return torch.tensor(idxs)
    return torch.tensor(idxs).cuda()
#     return torch.tensor(idxs).type(torch.cuda.FloatTensor)

In [13]:
def get_bert_vecs(tokens, args_in):
      
    text = ' '.join(tokens) 
    bert_tokens, orig_to_token_map = tokenization(tokens)
    segments_ids = [0 for i in range(len(bert_tokens))]
    indexed_tokens = tokenizer.convert_tokens_to_ids(bert_tokens)
    
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    encoded_layers, pooled_output = bert_model(tokens_tensor, segments_tensors, output_all_encoded_layers=False)
    
    vecs = encoded_layers[0].cuda()
    
    return vecs, orig_to_token_map

In [14]:
def get_dps(conll):
    dps = []
    for tok in conll:
        dp = tok[10]
        dps.append(dp)
    return dps

In [15]:
def get_labels_by_tensor(t):
    value, indices = t.max(1)
    score = pow(1, value)
    labels = []
    for i in indices:
        for label, idx in arg_to_ix.items():
            if idx == i:
                pred = label
                labels.append(pred)
                break
    return labels, score
            
        
    pred = None
    for label, idx in arg_to_ix.items():
        if idx == indices:
            pred = label
            break
    return pred, score



def eval_dev(my_model):
    sent, golds, preds = [],[],[]
    n = 0
    for s_idx in range(len(dev)):
        tokens, args = dev[s_idx][1], dev[s_idx][2]
        args_in = prepare_sequence(args, arg_to_ix)  

        conll = read_data.get_nlp_for_trn(s_idx+27884)
        dps = get_dps(conll)
        dp_in = prepare_sequence(dps, dp_to_ix)

        pred_idxs = get_pred_idxs(conll)
        features = get_feature(pred_idxs, conll)
        features = torch.tensor(features).type(torch.cuda.FloatTensor)  


        gold = args
        pred = ['-' for i in range(len(args))]
        for i in range(len(pred_idxs)):
            input_vec = []
            feat_vectors = features[i]
            bert_vecs, orig_to_token_map = get_bert_vecs(tokens,args_in)

            for tok_idx in range(len(feat_vectors)):
                add_feat = torch.cat((bert_vecs[orig_to_token_map[tok_idx]], feat_vectors[tok_idx]))

                input_vec.append(add_feat)
            input_vec = torch.stack(input_vec)           

            tag_scores = my_model(input_vec, dp_in)           
            labels, score = get_labels_by_tensor(tag_scores)
            
            for idx in range(len(labels)):
                label = labels[idx]
                if label == '-':
                    pass
                else:
                    if label[idx] == '-':
                        label[idx] = label
        for i in gold:
            if i == '-': i = 'O'
            else: i = 'S-'+i
        for i in pred:
            if i == '-': i = 'O'
            else: i = 'S-'+i
        
        sent.append(tokens)
        golds.append(gold)
        preds.append(pred)    
    
    result = []
    result.append(sent)
    result.append(golds)
    result.append(preds)
    
    f1 = f1_score(golds, preds)
    report = classification_report(golds, preds)
    
    with open(model_path+'/result','w') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)        
    return f1, report

# model

In [17]:
class LSTMTagger(nn.Module):
    
    def __init__(self, tagset_size):
        super(LSTMTagger, self).__init__()
        
        self.dp_embeddings = nn.Embedding(DP_VOCAB_SIZE, DPDIM)
        
        #LSTM layer        
        self.lstm_tok = nn.LSTM(LSTMINPDIM+DPDIM+FEATDIM, HIDDENDIM//2, bidirectional=True, num_layers=LSTMDEPTH, dropout=DROPOUT_RATE)
        self.hidden = self.init_hidden()
        
        # Linear
        self.hidden2tag = nn.Linear(HIDDENDIM, tagset_size)
              

    def init_hidden(self):
        return (torch.zeros(4, 1, HIDDENDIM//2).cuda(),
            torch.zeros(4, 1, HIDDENDIM//2).cuda())
    
    def forward(self, input_vecs, dp_in):
        
        dp_embs = self.dp_embeddings(dp_in)
        
        input_embs = torch.cat( (input_vecs, dp_embs), 1)        
        input_embs = input_embs.view(len(input_vecs), 1, -1)
        
        # LSTM layer
        lstm_out_tok, self.hidden = self.lstm_tok(
            input_embs, self.hidden)
        
        # Linear
        tag_space = self.hidden2tag(lstm_out_tok.view(len(input_vecs),-1))        
        tag_space = F.relu(tag_space)        
        softmax = nn.Softmax(dim=1)
        tag_scores = softmax(tag_space)
#         tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores       

In [25]:
srl_model = LSTMTagger(ARG_VOCAB_SIZE)
srl_model.cuda()
loss_function = nn.CrossEntropyLoss()
# loss_function = nn.NLLLoss()
optimizer = optim.Adam(srl_model.parameters(), lr=learning_rate)
# optimizer = optim.SGD(srl_model.parameters(), lr=learning_rate)

total_step = len(trn)
for epoch in range(NUM_EPOCHS):
    n_of_step = 0

    for s_idx in range(len(trn)):
        n_of_sent = 1
        tokens, args = trn[s_idx][1], trn[s_idx][2]
        args_in = prepare_sequence(args, arg_to_ix)  

        conll = read_data.get_nlp_for_trn(s_idx)
        dps = get_dps(conll)
        dp_in = prepare_sequence(dps, dp_to_ix)

        pred_idxs = get_pred_idxs(conll)

        for i in range(len(pred_idxs)):
            pred_seq = pred_idxs[i]
            
            for j in range(len(pred_seq)):
                p = pred_seq[j]
                if p == 1:
                    pred_idx = j
            arg_idxs = get_arg_idxs(pred_idx, conll)          
            print(arg_idxs)
            
        break
    break

#         for i in range(len(pred_idxs)):
#             input_vec = []
#             feat_vectors = features[i]
#             bert_vecs, orig_to_token_map = get_bert_vecs(tokens,args_in)



#             for tok_idx in range(len(feat_vectors)):
#                 add_feat = torch.cat((bert_vecs[orig_to_token_map[tok_idx]], feat_vectors[tok_idx]))

#                 input_vec.append(add_feat)
#             input_vec = torch.stack(input_vec)

#             srl_model.zero_grad()
#             srl_model.hidden = srl_model.init_hidden()

            

#             tag_scores = srl_model(input_vec, dp_in)
#             loss = loss_function(tag_scores, args_in)
#             loss.backward()
#             torch.nn.utils.clip_grad_norm_(srl_model.parameters(), 0.25)
#             optimizer.step()
        
#             n_of_step +=1

#             if n_of_step % 100 == 0:
#                 print('Epoch [{}/{}], Sent/Step [{}/{}], Loss: {:.4f}' 
#                        .format(epoch+1, NUM_EPOCHS, n_of_sent, n_of_step, loss.item()))
#         n_of_sent +=1
            
#     f1, report = eval_dev(srl_model)
#     print('Epoch [{}/{}], F1: {:4f}' 
#                    .format(epoch+1, NUM_EPOCHS, f1))

# print('### report ###')
# print(report)
        
# torch.save(srl_model, model_path)
# print('')
# print('### YOUR MODEL IS SAVED TO', model_path, '###')

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [67]:
golds = []
preds = []

for s_idx in range(len(dev)):
    n_of_sent = 1
    tokens, args = trn[s_idx][1], trn[s_idx][2]
    args_in = prepare_sequence(args, arg_to_ix)  
    conll = read_data.get_nlp_for_trn(s_idx)
    dps = get_dps(conll)
    dp_in = prepare_sequence(dps, dp_to_ix)
    pred_idxs = get_pred_idxs(conll)
    
    gold = args

    pred = ['O' for i in range(len(args))]
    for i in range(len(pred_idxs)):
        pred_seq = pred_idxs[i]

        
        
        
        for j in range(len(pred_seq)):
            p = pred_seq[j]
            if p == 1:
                pred_idx = j
        arg_idxs = get_arg_idxs(pred_idx, conll)
        
        labels = []
        for idx in arg_idxs:
            if idx == 1:
                label = 'S-ARG0'
            else:
                label = 'O'
            labels.append(label)
                
                
        for idx in range(len(labels)):
            label = labels[idx]
            if label == 'O':
                pass
            else:
                pred[idx] = label
#                 print('wow')
#                 print(label)
#                 print(pred[idx])
    
    gold = []
    for i in args:
        if i == '-': 
            g = 'O'
        else: 
            g = 'S-ARG0'
        gold.append(g)

    golds.append(gold)
    preds.append(pred)

#     print('gold')
#     print(gold)
#     print('pred')
#     print(pred)
#     print('')
#     break
        
    
f1 = f1_score(golds, preds)
a = accuracy_score(golds,preds)
report = classification_report(golds, preds)

print(f1)
print(a)
print(report)


0.8347807402367182
0.8879443304227439
             precision    recall  f1-score   support

       ARG0       0.75      0.94      0.83     24715

avg / total       0.75      0.94      0.83     24715



In [56]:
from sklearn.metrics import f1_score
n = 0
for i in range(len(golds)):
    gold = golds[i]
    pred = preds[i]
    f1 = f1 + f1_score(gold, pred, average='macro')
    
    n +=1
    
result = f1/n
print(result)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.27752060604123824
